# *To catch a protagonist* in DraCor
**by Ingo Börner**

In the paper [*To Catch a Protagonist: Quantitative Dominance Relations in German-Language Drama (1730-1930)*](https://dh2018.adho.org/to-catch-a-protagonist-quantitative-dominance-relations-in-german-language-drama-1730-1930/) (Fischer et al. 2018) an algorithm is described, that allows to identify characters that are the quantitatively dominant characters of a play based on a set of network-based and count based measures:

> In order to systematically describe the extent of this deviation, we calculate eight values for each character of the 465 dramas of our corpus, three count-based measures (number of scenes a character appears in, number of speech acts, number of spoken words) and five network-related measures (degree, weighted degree, betweenness centrality, closeness centrality, eigenvector centrality). For each measurement a ranking is created. The rankings are then merged into two meta-rankings: one count-based and one network-based. The two meta-rankings are then combined into an overall ranking.

The original algorithm was implemented in the tool [Dramavis](https://github.com/lehkost/dramavis) by Christopher Kittel. *Dramavis* operates on XML ["zwischenformat"](https://dlina.github.io/Introducing-Our-Zwischenformat) files created in the [DLINA](https://dlina.github.io) project. 

The following notebook adapts the code of the respective modules to work with data returned by the [DraCor API](https://dracor.org/doc/api). The aim is to be able to recreate the `*_chars.csv`-files that were used in the study. The data can be found in the [repository](https://github.com/dlina/catch-protagonist) on github in the folder [`allmetrics`](https://github.com/dlina/catch-protagonist/tree/main/data/allmetrics).

The implementation will be tested with the play *Emilia Galotti*. The original algorithm operated on the corresponding [LINA](https://dlina.github.io/linas/88) and produced the file [`88_Emilia Galotti_chars.csv`](https://github.com/dlina/catch-protagonist/blob/main/data/allmetrics/88_Emilia%20Galotti_chars.csv) as output In DraCor the play can be accessed [here](https://dracor.org/ger/lessing-emilia-galotti).


## Step 1. Get the basic measures

We need to get the following basic measures on characters:

**Network measures**

* betweenness
* degree
* closeness
* ~~closeness corrected~~
* weighted degree
* eigenvector centrality

**count-based measures**

* frequency/appearances
* number of *speech acts*
* number of *words*

### Network and count-based metrics via Dracor API

The Python-Packages [`requests`](https://docs.python-requests.org/en/latest/) and the library [`json`](https://docs.python.org/3/library/json.html#module-json) will be used to query the API and parse the response:

In [1]:
# if not installed, uncomment the following line and run the cell:
# !pip install requests

In [2]:
import requests
import json

In [3]:
# set corpus and playname
corpusname = "ger"
playname = "lessing-emilia-galotti"

In [4]:
# base url of the DraCor-API
api_base = "https://dracor.org/api/v1/"

To retrieve the network-data and speech-amounts data on single characters the function `/corpora/{corpusname}/play/{playname}/cast` is used as follows:

In [5]:
# send a request to the endpoint and parse results
request_url = api_base + "corpora/" + corpusname + "/plays/" + playname + "/characters"
character_data = requests.get(request_url).json()

The API function returns data on the characters, including the network and count-based metrics:

In [6]:
character_data

[{'numOfScenes': 17,
  'sex': 'MALE',
  'betweenness': 0.46717171717171724,
  'degree': 8,
  'numOfWords': 4002,
  'closeness': 0.75,
  'id': 'der_prinz',
  'numOfSpeechActs': 157,
  'name': 'Der Prinz',
  'isGroup': False,
  'eigenvector': 0.32076106311648156,
  'weightedDegree': 20},
 {'numOfScenes': 2,
  'sex': 'MALE',
  'betweenness': 0,
  'degree': 1,
  'numOfWords': 33,
  'closeness': 0.4444444444444444,
  'id': 'der_kammerdiener',
  'numOfSpeechActs': 6,
  'name': 'Der Kammerdiener',
  'isGroup': False,
  'eigenvector': 0.05575792046031641,
  'weightedDegree': 2},
 {'numOfScenes': 2,
  'sex': 'MALE',
  'betweenness': 0,
  'degree': 1,
  'numOfWords': 604,
  'closeness': 0.4444444444444444,
  'id': 'conti',
  'numOfSpeechActs': 24,
  'name': 'Conti',
  'isGroup': False,
  'eigenvector': 0.05575792046031641,
  'weightedDegree': 2},
 {'numOfScenes': 19,
  'sex': 'MALE',
  'betweenness': 0.24696969696969698,
  'degree': 9,
  'numOfWords': 4343,
  'closeness': 0.8,
  'id': 'marinelli

The data on the characters are in a dictionary:
```
{'betweenness': 0.24696969696969698,
  'closeness': 0.8,
  'degree': 9,
  'eigenvector': 0.44898463593218985,
  'gender': 'MALE',
  'id': 'marinelli',
  'isGroup': False,
  'name': 'Marinelli',
  'numOfScenes': 19,
  'numOfSpeechActs': 221,
  'numOfWords': 4343,
  'weightedDegree': 30}
```

We don't know anything about the network metrics of the whole play, though. If we want to retrieve this information, we would have to use the API function `/corpora/{corpusname}/play/{playname}/metrics`, which would also tell us, if there are several sub-networks in a dictionary-field with the key `numConnectedComponents`. This could be relevant, because we can also calculate some network-metrics differently, e.g. the *closeness*.

## Preparation: Get the metrics and construct a pandas data frame

In the Dramavis implementation an object of the class `DramaAnalyzer` is created, which contains the information on characters in a pandas data frame. We will create the same data structure to be able to use the same methods for calculating means and ranking.

The rows in the [table](https://raw.githubusercontent.com/dlina/catch-protagonist/main/data/allmetrics/88_Emilia%20Galotti_chars.csv) are:

`name,betweenness,degree,closeness,closeness_corrected,strength,eigenvector_centrality,avg_distance,avg_distance_corrected,frequency,speech_acts,words,lines,chars ...`

We will not include all rows, but only the ones, that are relevant for the rankings:

`name`,`betweenness`,`degree`,`closeness`,~~closeness_corrected~~,`strength`,`eigenvector_centrality`,~~avg_distance,avg_distance_corrected~~,`frequency`,`speech_acts`,`words`,~~lines,chars~~ ...

following rows will be called differently to follow DraCor conventions of the [API output](https://dracor.org/api/corpora/ger/play/lessing-emilia-galotti/cast):

* `name` → `id`; later this will be used to construct URIs
* `strength` → `weightedDegree`
* `eigenvector_centrality` → `eigenvector`
* `frequency` → `numOfScenes`
* `speech_acts` → `numOfSpeechActs`
* `words` → `numOfWords`



The package [`pandas`](https://pandas.pydata.org/) is used to handle the data as a dataframe. Therefore we need to import the package.

In [7]:
# if not installed, uncomment the following line and run the cell:
# !pip install pandas

In [8]:
import pandas as pd

In [9]:
# construct the data frame
df = pd.DataFrame(character_data)

#turn the column "id" to the index
df = df.set_index('id')     

In [10]:
#output
df

,numOfScenes,sex,betweenness,degree,numOfWords,closeness,numOfSpeechActs,name,isGroup,eigenvector,weightedDegree
id,,,,,,,,,,,
der_prinz,17,MALE,0.467172,8,4002,0.750000,157,Der Prinz,False,0.320761,20
der_kammerdiener,2,MALE,0.000000,1,33,0.444444,6,Der Kammerdiener,False,0.055758,2
conti,2,MALE,0.000000,1,604,0.444444,24,Conti,False,0.055758,2
marinelli,19,MALE,0.246970,9,4343,0.800000,221,Marinelli,False,0.448985,30
camillo_rota,1,MALE,0.000000,1,78,0.444444,6,Camillo Rota,False,0.055758,1
claudia,13,FEMALE,0.045455,7,1581,0.600000,73,Claudia,False,0.382926,19
pirro,4,MALE,0.026515,5,263,0.545455,25,Pirro,False,0.271944,7
odoardo,12,MALE,0.055051,6,2441,0.666667,108,Odoardo,False,0.354250,15
angelo,2,MALE,0.000000,2,487,0.480000,28,Angelo,False,0.125318,2


We can now query the data, e.g. output the values of a single character by requesting a row by its index value, which is the `id` of the character. 

In [11]:
# get the values of a single character
df.loc["marinelli"]

numOfScenes               19
sex                     MALE
betweenness          0.24697
degree                     9
numOfWords              4343
closeness                0.8
numOfSpeechActs          221
name               Marinelli
isGroup                False
eigenvector         0.448985
weightedDegree            30
Name: marinelli, dtype: object

## Step 2. Calculate the ranks

In Dramavis the function [`get_character_ranks`](https://github.com/lehkost/dramavis/blob/v0.4/dramalyzer.py#L282-L288) creates the rankings of the count-based and network-based measures. We will adapt this function to operate on the created data frame and rename the columns:

In [12]:
metrics_to_rank = ['degree', 'closeness', 'betweenness', 'weightedDegree', 'eigenvector', 'numOfScenes', 'numOfSpeechActs', 'numOfWords']
for metric in metrics_to_rank:
    df[metric + "_rank"] = df[metric].rank(method='min', ascending=False)
df
    

,numOfScenes,sex,betweenness,degree,numOfWords,closeness,numOfSpeechActs,name,isGroup,eigenvector,weightedDegree,degree_rank,closeness_rank,betweenness_rank,weightedDegree_rank,eigenvector_rank,numOfScenes_rank,numOfSpeechActs_rank,numOfWords_rank
id,,,,,,,,,,,,,,,,,,,
der_prinz,17,MALE,0.467172,8,4002,0.750000,157,Der Prinz,False,0.320761,20,2.0,2.0,1.0,2.0,5.0,2.0,2.0,2.0
der_kammerdiener,2,MALE,0.000000,1,33,0.444444,6,Der Kammerdiener,False,0.055758,2,11.0,11.0,10.0,10.0,11.0,10.0,12.0,13.0
conti,2,MALE,0.000000,1,604,0.444444,24,Conti,False,0.055758,2,11.0,11.0,10.0,10.0,11.0,10.0,10.0,8.0
marinelli,19,MALE,0.246970,9,4343,0.800000,221,Marinelli,False,0.448985,30,1.0,1.0,2.0,1.0,1.0,1.0,1.0,1.0
camillo_rota,1,MALE,0.000000,1,78,0.444444,6,Camillo Rota,False,0.055758,1,11.0,11.0,10.0,13.0,11.0,13.0,12.0,12.0
claudia,13,FEMALE,0.045455,7,1581,0.600000,73,Claudia,False,0.382926,19,3.0,5.0,5.0,3.0,2.0,3.0,4.0,6.0
pirro,4,MALE,0.026515,5,263,0.545455,25,Pirro,False,0.271944,7,6.0,8.0,6.0,8.0,6.0,8.0,9.0,10.0
odoardo,12,MALE,0.055051,6,2441,0.666667,108,Odoardo,False,0.354250,15,4.0,3.0,3.0,4.0,3.0,4.0,3.0,3.0
angelo,2,MALE,0.000000,2,487,0.480000,28,Angelo,False,0.125318,2,10.0,10.0,10.0,10.0,10.0,10.0,8.0,9.0


## Step 3. Rank on average and standard deviation of the individual rankings

In Dramavis the individual rankings are then used for the calculation of an average ranking and the standard deviation, which are then also ranked. This is done by the function [`get_centrality_ranks`](https://github.com/lehkost/dramavis/blob/v0.4/dramalyzer.py#L305-L310).

The following columns will be added to the data frame:

* (1) `centrality_rank_avg`: The average of all rankings
* (2) `centrality_rank_std`: Standard deviation of the rankings
* (3) `centrality_rank_avg_rank`: A ranking is created from the average of all rankings (1)
* (4) `centrality_rank_std_rank`: A ranking is created from the standard deviation of all rankings (2)

The following dramavis code is adapted accordingly to operate on the dataframe:

In [13]:
ranks = [c for c in df.columns if c.endswith("rank")]
df['centrality_rank_avg'] = df[ranks].sum(axis=1)/len(ranks)
df['centrality_rank_std'] = df[ranks].std(axis=1)/len(ranks)
for metric in ['centrality_rank_avg', 'centrality_rank_std']:
    df[metric + "_rank"] = df[metric].rank(method='min', ascending=True)
df

,numOfScenes,sex,betweenness,degree,numOfWords,closeness,numOfSpeechActs,name,isGroup,eigenvector,...,betweenness_rank,weightedDegree_rank,eigenvector_rank,numOfScenes_rank,numOfSpeechActs_rank,numOfWords_rank,centrality_rank_avg,centrality_rank_std,centrality_rank_avg_rank,centrality_rank_std_rank
id,,,,,,,,,,,,,,,,,,,,,
der_prinz,17,MALE,0.467172,8,4002,0.750000,157,Der Prinz,False,0.320761,...,1.0,2.0,5.0,2.0,2.0,2.0,2.250,0.145621,2.0,9.0
der_kammerdiener,2,MALE,0.000000,1,33,0.444444,6,Der Kammerdiener,False,0.055758,...,10.0,10.0,11.0,10.0,12.0,13.0,11.000,0.133631,12.0,7.0
conti,2,MALE,0.000000,1,604,0.444444,24,Conti,False,0.055758,...,10.0,10.0,11.0,10.0,10.0,8.0,10.125,0.123879,11.0,5.0
marinelli,19,MALE,0.246970,9,4343,0.800000,221,Marinelli,False,0.448985,...,2.0,1.0,1.0,1.0,1.0,1.0,1.125,0.044194,1.0,1.0
camillo_rota,1,MALE,0.000000,1,78,0.444444,6,Camillo Rota,False,0.055758,...,10.0,13.0,11.0,13.0,12.0,12.0,11.625,0.132583,13.0,6.0
claudia,13,FEMALE,0.045455,7,1581,0.600000,73,Claudia,False,0.382926,...,5.0,3.0,2.0,3.0,4.0,6.0,3.875,0.169525,4.0,11.0
pirro,4,MALE,0.026515,5,263,0.545455,25,Pirro,False,0.271944,...,6.0,8.0,6.0,8.0,9.0,10.0,7.625,0.188243,7.0,12.0
odoardo,12,MALE,0.055051,6,2441,0.666667,108,Odoardo,False,0.354250,...,3.0,4.0,3.0,4.0,3.0,3.0,3.375,0.064694,3.0,2.0
angelo,2,MALE,0.000000,2,487,0.480000,28,Angelo,False,0.125318,...,10.0,10.0,10.0,10.0,8.0,9.0,9.625,0.093003,10.0,3.0


Based on the calculation of `centrality_rank_avg_rank`, the "central" characters can be already queried as follows:

In [14]:
df[df["centrality_rank_avg_rank"] == 1].index.tolist()

['marinelli']

## Additional Step: Create Rankings and combined rankings of network-based and count-based metrics separately

In addition to a ranking that combines all metrics and rankings derived thereof, the function [`get_structural_ranking_measures`](https://github.com/lehkost/dramavis/blob/v0.4/dramalyzer.py#L318-L330) treats network-based and count-based values separately and only then aggregates them to a combined overall ranking.

The function adds the following rows to the data frame:

* (1) `avg_graph_rank`: a ranking based on the rankings of the network-values (*degree*, *closeness*, *betweenness*, *strength* or *weightedDegree* and *eigenvector centrality* or *eigenvector*)
* (2) `avg_content_rank`: a ranking based on the rankings of the count-based values (*frequency* or *numOfScenes*, *speech acts* and *words*)
* (3) `overall_avg`: the two rankings (1+2) are combined by calculating the mean
* (4) `overall_avg_rank`: based on the overall average (3) a ranking is created

The following code is adapted accordingly to operate on the dataframe. The ranking stability measures are not implemented here.

In [15]:
#renamed the columns to match the DraCor values here:
graph_ranks = ['degree_rank', 'closeness_rank', 'betweenness_rank', 'weightedDegree_rank', 'eigenvector_rank']
content_ranks = ['numOfScenes_rank', 'numOfSpeechActs_rank', 'numOfWords_rank']
avg_graph_rank = df[graph_ranks].mean(axis=1).rank(method='min')
avg_content_rank = df[content_ranks].mean(axis=1).rank(method='min')
df["avg_graph_rank"] = avg_graph_rank
df["avg_content_rank"] = avg_content_rank
df["overall_avg"] = df[["avg_graph_rank", "avg_content_rank"]].mean(axis=1)
df["overall_avg_rank"] = df["overall_avg"].rank(method='min')
df

,numOfScenes,sex,betweenness,degree,numOfWords,closeness,numOfSpeechActs,name,isGroup,eigenvector,...,numOfSpeechActs_rank,numOfWords_rank,centrality_rank_avg,centrality_rank_std,centrality_rank_avg_rank,centrality_rank_std_rank,avg_graph_rank,avg_content_rank,overall_avg,overall_avg_rank
id,,,,,,,,,,,,,,,,,,,,,
der_prinz,17,MALE,0.467172,8,4002,0.750000,157,Der Prinz,False,0.320761,...,2.0,2.0,2.250,0.145621,2.0,9.0,2.0,2.0,2.0,2.0
der_kammerdiener,2,MALE,0.000000,1,33,0.444444,6,Der Kammerdiener,False,0.055758,...,12.0,13.0,11.000,0.133631,12.0,7.0,11.0,12.0,11.5,12.0
conti,2,MALE,0.000000,1,604,0.444444,24,Conti,False,0.055758,...,10.0,8.0,10.125,0.123879,11.0,5.0,11.0,10.0,10.5,11.0
marinelli,19,MALE,0.246970,9,4343,0.800000,221,Marinelli,False,0.448985,...,1.0,1.0,1.125,0.044194,1.0,1.0,1.0,1.0,1.0,1.0
camillo_rota,1,MALE,0.000000,1,78,0.444444,6,Camillo Rota,False,0.055758,...,12.0,12.0,11.625,0.132583,13.0,6.0,13.0,13.0,13.0,13.0
claudia,13,FEMALE,0.045455,7,1581,0.600000,73,Claudia,False,0.382926,...,4.0,6.0,3.875,0.169525,4.0,11.0,4.0,4.0,4.0,4.0
pirro,4,MALE,0.026515,5,263,0.545455,25,Pirro,False,0.271944,...,9.0,10.0,7.625,0.188243,7.0,12.0,7.0,8.0,7.5,7.0
odoardo,12,MALE,0.055051,6,2441,0.666667,108,Odoardo,False,0.354250,...,3.0,3.0,3.375,0.064694,3.0,2.0,3.0,3.0,3.0,3.0
angelo,2,MALE,0.000000,2,487,0.480000,28,Angelo,False,0.125318,...,8.0,9.0,9.625,0.093003,10.0,3.0,10.0,8.0,9.0,9.0


Based on the calculation of `overall_avg_rank`, the "central" characters can be queried as follows:

In [16]:
df[df["overall_avg_rank"] == 1].index.tolist()

['marinelli']